# Libraries 

In [1]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import pandas as pd

import numpy as np

import matplotlib as mpl

import matplotlib.pyplot as plt

import seaborn as sns 

sns.set(color_codes = True)

%matplotlib inline

import plotly.graph_objects as go

from plotly.subplots import make_subplots

from sklearn.feature_extraction.text import CountVectorizer

from collections import Counter

from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

from textblob import Word

from nltk.stem import WordNetLemmatizer

from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score, KFold

from textblob import TextBlob

import gender_guesser.detector as gender

import nltk

from nltk.corpus import stopwords

import re

import emoji

from scipy.stats import chi2_contingency

from matplotlib.patches import ConnectionPatch

from langdetect import detect

from nltk.probability import *

from nltk.tokenize import word_tokenize

from nltk import pos_tag

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer

import plotly.graph_objects as go

from plotly.subplots import make_subplots

from afinn import Afinn

from sklearn.model_selection import cross_val_score, KFold

from sklearn.ensemble import RandomForestClassifier

from nltk.text import Text

import string

from collections import Counter

from sklearn.naive_bayes import MultinomialNB

from sklearn.linear_model import LogisticRegression

from sklearn.svm import LinearSVC

from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

In [3]:
ml = pd.read_csv("ml.csv")

In [4]:
ml.dropna(subset = ["cleaned_text"], inplace = True)

In [5]:
ml.tail(1)

,starRating,Gender_Numeric,Year,cleaned_text,sentiment_score,word_count,avg_word,sentiment_label,sentiment_ml,sentiment
1647,5,1,2018,good,0.7,1,4.0,Positive,2,0


# N-grams

In [6]:
X = ml["cleaned_text"]
y = ml["sentiment_ml"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, 
                                                    random_state = 12, stratify = y)

ngram_ranges = [(1, 1), (1, 2), (1, 3)]

for ngram_range in ngram_ranges:
    print(f"Testing ngram_range: {ngram_range}")
    
    tfidf = TfidfVectorizer(ngram_range = ngram_range, max_features = 500, tokenizer = word_tokenize)
    X_train_tfidf = tfidf.fit_transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)

    models = [
        ('RF', RandomForestClassifier()),
        ('MNB', MultinomialNB()),
        ('XGB', XGBClassifier()),
        ('LR', LogisticRegression(max_iter = 500)),
        ('LSVC', LinearSVC())
    ]

    class_reports = {}
    for name, model in models:
        kfold = StratifiedKFold(n_splits = 10, random_state = 12, shuffle = True)
        cv_results = cross_val_score(model, X_train_tfidf, y_train, cv = kfold, scoring = 'accuracy')
        print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")

        model.fit(X_train_tfidf, y_train)
        y_pred = model.predict(X_test_tfidf)
        class_reports[name] = classification_report(y_test, y_pred)

    for name, report in class_reports.items():
        print(f"\nClassification Report for {name}:\n{report}\n")


Testing ngram_range: (1, 1)
RF: Mean Accuracy - 0.8412, Std. Deviation - 0.0313
MNB: Mean Accuracy - 0.7892, Std. Deviation - 0.0236
XGB: Mean Accuracy - 0.8547, Std. Deviation - 0.0384
LR: Mean Accuracy - 0.8561, Std. Deviation - 0.0330
LSVC: Mean Accuracy - 0.8601, Std. Deviation - 0.0343

Classification Report for RF:
              precision    recall  f1-score   support

           0       1.00      0.14      0.25         7
           1       0.79      0.87      0.83        70
           2       0.90      0.89      0.89        88

    accuracy                           0.85       165
   macro avg       0.90      0.63      0.66       165
weighted avg       0.86      0.85      0.84       165



Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.80      0.61      0.69        70
           2       0.75      0.94      0.83        88

    accuracy                           0.76  

# Test Size

In [7]:
X = ml["cleaned_text"]
y = ml["sentiment_ml"]

test_sizes = [0.1, 0.2, 0.3, 0.4]
ngram_range = (1, 1)

for test_size in test_sizes:
    print(f"Testing test_size: {test_size}")
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, 
                                                        random_state = 12, stratify = y)

    tfidf = TfidfVectorizer(ngram_range = ngram_range, max_features = 500, tokenizer = word_tokenize)
    X_train_tfidf = tfidf.fit_transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)

    models = [
        ('RF', RandomForestClassifier()),
        ('MNB', MultinomialNB()),
        ('XGB', XGBClassifier()),
        ('LR', LogisticRegression(max_iter = 500)),
        ('LSVC', LinearSVC())
    ]

    class_reports = {}
    for name, model in models:
        kfold = StratifiedKFold(n_splits = 10, random_state = 12, shuffle = True)
        cv_results = cross_val_score(model, X_train_tfidf, y_train, cv = kfold, scoring = 'accuracy')
        print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")

        model.fit(X_train_tfidf, y_train)
        y_pred = model.predict(X_test_tfidf)
        class_reports[name] = classification_report(y_test, y_pred)

    for name, report in class_reports.items():
        print(f"\nClassification Report for {name}:\n{report}\n")


Testing test_size: 0.1
RF: Mean Accuracy - 0.8338, Std. Deviation - 0.0320
MNB: Mean Accuracy - 0.7892, Std. Deviation - 0.0236
XGB: Mean Accuracy - 0.8547, Std. Deviation - 0.0384
LR: Mean Accuracy - 0.8561, Std. Deviation - 0.0330
LSVC: Mean Accuracy - 0.8601, Std. Deviation - 0.0343

Classification Report for RF:
              precision    recall  f1-score   support

           0       1.00      0.14      0.25         7
           1       0.79      0.90      0.84        70
           2       0.92      0.88      0.90        88

    accuracy                           0.85       165
   macro avg       0.90      0.64      0.66       165
weighted avg       0.87      0.85      0.84       165



Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.80      0.61      0.69        70
           2       0.75      0.94      0.83        88

    accuracy                           0.76       

# Under Sampling

In [8]:
from imblearn.under_sampling import NearMiss

X = ml["cleaned_text"]
y = ml["sentiment_ml"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 12, stratify = y)

tfidf = TfidfVectorizer(ngram_range = (1, 1), max_features = 500, tokenizer = word_tokenize)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

nr = NearMiss()
X_train_nm, y_train_nm = nr.fit_resample(X_train_tfidf, y_train)

models = [
    ('RF', RandomForestClassifier()),
    ('MNB', MultinomialNB()),
    ('XGB', XGBClassifier()),
    ('LR', LogisticRegression(max_iter = 500)),
    ('LSVC', LinearSVC())
]


class_reports = {}
for name, model in models:
    kfold = StratifiedKFold(n_splits = 10, random_state = 12, shuffle = True)
    cv_results = cross_val_score(model, X_train_nm, y_train_nm, cv = kfold, scoring = 'accuracy')
    print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")
    
    model.fit(X_train_nm, y_train_nm)
    y_pred = model.predict(X_test_tfidf)
    class_reports[name] = classification_report(y_test, y_pred)

for name, report in class_reports.items():
    print(f"\nClassification Report for {name}:\n{report}\n")

RF: Mean Accuracy - 0.7132, Std. Deviation - 0.0630
MNB: Mean Accuracy - 0.6574, Std. Deviation - 0.1328
XGB: Mean Accuracy - 0.7182, Std. Deviation - 0.0558
LR: Mean Accuracy - 0.6826, Std. Deviation - 0.0395
LSVC: Mean Accuracy - 0.6405, Std. Deviation - 0.0627

Classification Report for RF:
              precision    recall  f1-score   support

           0       0.33      0.57      0.42         7
           1       0.63      0.74      0.68        70
           2       0.83      0.67      0.74        88

    accuracy                           0.70       165
   macro avg       0.60      0.66      0.62       165
weighted avg       0.73      0.70      0.70       165



Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.36      0.71      0.48         7
           1       0.82      0.46      0.59        70
           2       0.72      0.92      0.81        88

    accuracy                           0.72       165
   macro avg       

# Over Sampling

In [9]:
from imblearn.over_sampling import SMOTE

X = ml["cleaned_text"]
y = ml["sentiment_ml"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 12, stratify = y)

tfidf = TfidfVectorizer(ngram_range = (1, 1), max_features = 500, tokenizer = word_tokenize)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

smt = SMOTE()
X_train_s, y_train_s = nr.fit_resample(X_train_tfidf, y_train)

models = [
    ('RF', RandomForestClassifier()),
    ('MNB', MultinomialNB()),
    ('XGB', XGBClassifier()),
    ('LR', LogisticRegression(max_iter = 500)),
    ('LSVC', LinearSVC())
]


class_reports = {}
for name, model in models:
    kfold = StratifiedKFold(n_splits = 10, random_state = 12, shuffle = True)
    cv_results = cross_val_score(model, X_train_s, y_train_s, cv = kfold, scoring = 'accuracy')
    print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")
    
    model.fit(X_train_nm, y_train_nm)
    y_pred = model.predict(X_test_tfidf)
    class_reports[name] = classification_report(y_test, y_pred)

for name, report in class_reports.items():
    print(f"\nClassification Report for {name}:\n{report}\n")

RF: Mean Accuracy - 0.6926, Std. Deviation - 0.0669
MNB: Mean Accuracy - 0.6574, Std. Deviation - 0.1328
XGB: Mean Accuracy - 0.7182, Std. Deviation - 0.0558
LR: Mean Accuracy - 0.6826, Std. Deviation - 0.0395
LSVC: Mean Accuracy - 0.6405, Std. Deviation - 0.0627

Classification Report for RF:
              precision    recall  f1-score   support

           0       0.36      0.57      0.44         7
           1       0.68      0.67      0.68        70
           2       0.81      0.78      0.80        88

    accuracy                           0.73       165
   macro avg       0.62      0.68      0.64       165
weighted avg       0.74      0.73      0.73       165



Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.36      0.71      0.48         7
           1       0.82      0.46      0.59        70
           2       0.72      0.92      0.81        88

    accuracy                           0.72       165
   macro avg       

In [10]:
ml.tail(1)

,starRating,Gender_Numeric,Year,cleaned_text,sentiment_score,word_count,avg_word,sentiment_label,sentiment_ml,sentiment
1647,5,1,2018,good,0.7,1,4.0,Positive,2,0


In [11]:
from sklearn.neighbors import KNeighborsClassifier


X = ml["cleaned_text"]
y = ml["sentiment_ml"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, 
                                                    random_state = 12, stratify = y)

ngram_ranges = [(1, 1), (1, 2), (1, 3)]

for ngram_range in ngram_ranges:
    print(f"Testing ngram_range: {ngram_range}")
    
    tfidf = TfidfVectorizer(ngram_range=ngram_range, max_features = 500, tokenizer = word_tokenize)
    X_train_tfidf = tfidf.fit_transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)

    models = [
        ('RF', RandomForestClassifier()),
        ('MNB', MultinomialNB()),
        ('XGB', XGBClassifier()),
        ('LR', LogisticRegression(max_iter = 500)),
        ('LSVC', LinearSVC()),
        ("KNN", KNeighborsClassifier())
    ]

    class_reports = {}
    for name, model in models:
        kfold = StratifiedKFold(n_splits = 10, random_state = 12, shuffle = True)
        cv_results = cross_val_score(model, X_train_tfidf, y_train, cv = kfold, scoring = 'accuracy')
        print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")

        model.fit(X_train_tfidf, y_train)
        y_pred = model.predict(X_test_tfidf)
        class_reports[name] = classification_report(y_test, y_pred)

    for name, report in class_reports.items():
        print(f"\nClassification Report for {name}:\n{report}\n")


Testing ngram_range: (1, 1)
RF: Mean Accuracy - 0.8399, Std. Deviation - 0.0385
MNB: Mean Accuracy - 0.7892, Std. Deviation - 0.0236
XGB: Mean Accuracy - 0.8547, Std. Deviation - 0.0384
LR: Mean Accuracy - 0.8561, Std. Deviation - 0.0330
LSVC: Mean Accuracy - 0.8601, Std. Deviation - 0.0343
KNN: Mean Accuracy - 0.6101, Std. Deviation - 0.0877

Classification Report for RF:
              precision    recall  f1-score   support

           0       1.00      0.29      0.44         7
           1       0.80      0.87      0.84        70
           2       0.90      0.89      0.89        88

    accuracy                           0.85       165
   macro avg       0.90      0.68      0.72       165
weighted avg       0.86      0.85      0.85       165



Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.80      0.61      0.69        70
           2       0.75      0.94      0.83    

In [12]:
X = ml["cleaned_text"]
y = ml["sentiment_ml"]

test_sizes = [0.1, 0.2, 0.3, 0.4]
ngram_range = (1, 1)

for test_size in test_sizes:
    print(f"Testing test_size: {test_size}")
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, 
                                                        random_state = 12, stratify = y)

    tfidf = TfidfVectorizer(ngram_range = ngram_range, max_features = 2000, tokenizer = word_tokenize)
    X_train_tfidf = tfidf.fit_transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)

    models = [
        ('RF', RandomForestClassifier()),
        ('MNB', MultinomialNB()),
        ('XGB', XGBClassifier()),
        ('LR', LogisticRegression(max_iter = 500)),
        ('LSVC', LinearSVC())
    ]

    class_reports = {}
    for name, model in models:
        kfold = StratifiedKFold(n_splits = 10, random_state = 12, shuffle = True)
        cv_results = cross_val_score(model, X_train_tfidf, y_train, cv = kfold, scoring = 'accuracy')
        print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")

        model.fit(X_train_tfidf, y_train)
        y_pred = model.predict(X_test_tfidf)
        class_reports[name] = classification_report(y_test, y_pred)

    for name, report in class_reports.items():
        print(f"\nClassification Report for {name}:\n{report}\n")


Testing test_size: 0.1
RF: Mean Accuracy - 0.8412, Std. Deviation - 0.0326
MNB: Mean Accuracy - 0.7750, Std. Deviation - 0.0287
XGB: Mean Accuracy - 0.8601, Std. Deviation - 0.0414
LR: Mean Accuracy - 0.8547, Std. Deviation - 0.0295
LSVC: Mean Accuracy - 0.8561, Std. Deviation - 0.0284

Classification Report for RF:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.81      0.83      0.82        70
           2       0.87      0.92      0.90        88

    accuracy                           0.84       165
   macro avg       0.56      0.58      0.57       165
weighted avg       0.81      0.84      0.82       165



Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.82      0.59      0.68        70
           2       0.73      0.95      0.83        88

    accuracy                           0.76       